# Brief Analysis Example Using Transformed Data

This analysis uses the functions defined in the project, however it could also be valuable to test these same statistics on the processed data loaded into a local PostgreSQL database table. Connecting to this table could also lead to more meaningful, real-time analysis on the use of all the bike stations in the dataset.

In [15]:
# Import dependencies
import polars as pl
import sys
from pathlib import Path

_root = Path(".").resolve().parent
if str(_root) not in sys.path:
    sys.path.insert(0, str(_root))

from src.ingest import fetch_citybike_data
from src.transform import transform

In [16]:
# Transform current CityBike API data
df = transform(fetch_citybike_data())

Transforming the data...
Data transformed successfully.


## Summary: station count and average availability

In [17]:
print(f"Stations: {len(df)}")
print(f"Avg availability: {df['availability_pct'].mean():.0f}%")
df.select(pl.len().alias("n_stations"), pl.col("availability_pct").mean().alias("avg_availability_pct"))

Stations: 544
Avg availability: 49%


n_stations,avg_availability_pct
u32,f64
544,49.490809


## Which stations have the highest and lowest bike availability?

In [18]:
df_avail = df.sort("availability_pct", descending=True).select("name", "free_bikes", "total_docks", "availability_pct")
print(f"Station with highest availability: {df_avail.head(1)}")
print(f"Station with lowest availability: {df_avail.tail(1)}")

Station with highest availability: shape: (1, 4)
┌─────────────────┬────────────┬─────────────┬──────────────────┐
│ name            ┆ free_bikes ┆ total_docks ┆ availability_pct │
│ ---             ┆ ---        ┆ ---         ┆ ---              │
│ str             ┆ i64        ┆ i64         ┆ i64              │
╞═════════════════╪════════════╪═════════════╪══════════════════╡
│ 40 Thorndike St ┆ 19         ┆ 19          ┆ 100              │
└─────────────────┴────────────┴─────────────┴──────────────────┘
Station with lowest availability: shape: (1, 4)
┌─────────────────────────────────┬────────────┬─────────────┬──────────────────┐
│ name                            ┆ free_bikes ┆ total_docks ┆ availability_pct │
│ ---                             ┆ ---        ┆ ---         ┆ ---              │
│ str                             ┆ i64        ┆ i64         ┆ i64              │
╞═════════════════════════════════╪════════════╪═════════════╪══════════════════╡
│ Watermark Seaport - Boston Wh

## Stations below 30% availability
Candidates for rebalancing

In [19]:
df.filter(pl.col("availability_pct") < 30).select("name", "free_bikes", "total_docks", "availability_pct")

name,free_bikes,total_docks,availability_pct
str,i64,i64,i64
"""300 Summer St""",0,15,0
"""325 Binney St.""",3,19,15
"""555 Metropolitan Ave""",3,16,18
"""606 American Legion Hwy at Can…",4,18,22
"""615 Broadway""",0,11,0
…,…,…,…
"""W Broadway at Dorchester St""",0,16,0
"""Walnut Ave at School St""",0,19,0
"""Washington St at Temple Pl""",0,18,0


## Which stations have the highest and lowest total number of bikes/docks?

In [20]:
df_bikes = df.sort("total_docks", descending=True).select("name", "total_docks")
print(f"Station with highest amount of bikes: {df_bikes.head(1)}")
print(f"Station with lowest amount of bikes: {df_bikes.tail(1)}")

Station with highest amount of bikes: shape: (1, 2)
┌───────────────┬─────────────┐
│ name          ┆ total_docks │
│ ---           ┆ ---         │
│ str           ┆ i64         │
╞═══════════════╪═════════════╡
│ MIT Vassar St ┆ 53          │
└───────────────┴─────────────┘
Station with lowest amount of bikes: shape: (1, 2)
┌──────────────────────┬─────────────┐
│ name                 ┆ total_docks │
│ ---                  ┆ ---         │
│ str                  ┆ i64         │
╞══════════════════════╪═════════════╡
│ Fields Corner T Stop ┆ 8           │
└──────────────────────┴─────────────┘
